In [6]:
# pipelining

In [7]:
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import warnings
from sklearn.model_selection import GridSearchCV

warnings.filterwarnings('ignore')  # For clean output

# Load Titanic dataset
df = sns.load_dataset('titanic')

# Drop rows with missing target
df = df.dropna(subset=['survived'])

# Target and features
X = df.drop(columns=['survived'])
y = df['survived']

# Drop high-cardinality or unneeded cols
X = X.drop(columns=['embark_town', 'deck', 'alive', 'who', 'class'])

# Identify column types
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
numeric_cols = X.select_dtypes(include=['number', 'bool']).columns.tolist()

print("Categorical Columns:", categorical_cols)
print("Numeric Columns:", numeric_cols)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 1️⃣ Numeric Pipeline: Impute + Scale
numeric_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# 2️⃣ Categorical Pipeline: Impute + One-Hot Encode
categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# 3️⃣ Combine using ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_pipeline, numeric_cols),
    ('cat', categorical_pipeline, categorical_cols)
])

# 4️⃣ Final Pipeline: Preprocessor + Model
clf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Fit model
clf_pipeline.fit(X_train, y_train)

# Predict and evaluate
y_pred = clf_pipeline.predict(X_test)
print("\n✅ Classification Report:\n", classification_report(y_test, y_pred))


Categorical Columns: ['sex', 'embarked']
Numeric Columns: ['pclass', 'age', 'sibsp', 'parch', 'fare', 'adult_male', 'alone']

✅ Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.87      0.85       105
           1       0.80      0.76      0.78        74

    accuracy                           0.82       179
   macro avg       0.82      0.81      0.81       179
weighted avg       0.82      0.82      0.82       179



In [9]:
# Define hyperparameter grid (prefix with 'classifier__')
param_grid = {
    'classifier__n_estimators': [50, 100],
    'classifier__max_depth': [5, 10, None],
    'classifier__min_samples_split': [2, 5]
}

# GridSearchCV
grid_search = GridSearchCV(clf_pipeline, param_grid, cv=5, scoring='accuracy', verbose=2, n_jobs=-1)

# Fit
grid_search.fit(X_train, y_train)

# Results
print("\n✅ Best Parameters:")
print(grid_search.best_params_)

Fitting 5 folds for each of 12 candidates, totalling 60 fits

✅ Best Parameters:
{'classifier__max_depth': 5, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 50}
